<img src="PillowLogo.jpg" width="400">

# Hands-On Lab: Home Price Estimation

## Introduction
In this lab, PACCAR is entering the real estate market and taking on Zillow! We will be estimating home prices, showing how good data and good science get us the best recommendations. Specifically, we'll walk through:
* A simple linear regression
* AutoML
* Data understanding & feature engineering

In [ ]:
# Import any packages we will need to use for the analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import yaml
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import power_transform

from scipy import stats

## Import Data
When working in code, we first need to import our data. This can come from databases, spreadsheets, websites, etc.

```
[!] It is important to remember that data is loaded into your computer's memory. When people talk about "Big Data", it is commonly describing datasets that don't fit into memory!
```

In [ ]:
# Read in our dataset and investigate the first several rows
home_data = pd.read_csv("../Data/Train.csv")
home_data.head()

We should start by getting familiar with the dataset. Some common questions (and answers):

In [ ]:
# What is our target variable?
print("SalePrice")
home_data.SalePrice.head(3)

In [ ]:
# How many rows of data do we have?
print('Number of rows: {}'.format(len(home_data)))

In [ ]:
# How many attributes do we have for each row of data? What are the features?
print('Number of columns: {}'.format(home_data.shape[1]))
print(list(home_data))

Thankfully, the city of Ames, Iowa does a terrific job of cataloging data. Have a look at *./Documentation/data_description.txt* for more information about what these inputs mean. Unfortunately, this is a rare treat but a state we should strive for.

In [ ]:
# What data types are the features?
home_data.columns.to_series().groupby(home_data.dtypes).groups

## Measuring Quality
A strong suite of validation measures is critical to the success of an ML deployment.

For home pricing, we will use RMSE- Root Mean Square Error. 
<img src="rmse.jpg" width="400">
RMSE is commonly used in regression. We often communicate MAE (Mean Absolute Error), as it is more intuitive, but RMSE generally makes a better measure for maximizing model performance. The reason why is in the square term. RMSE penalizes larger errors more severely than smaller errors. 

For home pricing, we apply a small variation on RMSE, which is to measure the difference in the Logs of the prices. This is to assign equal weight to lower and higher priced homes.

```
[!] Plot Twist: This dataset comes from a public competition, for which there is a leaderboard! Throughout the lab, we will compare our performance against the leaderboard.
```

Our goal is to medal. Let's see what it takes!
<img src="medals.jpg" width="600">

In [ ]:
# Define RMSE formula to be used later
def getRMSE(data, target, prediction):
    data['num'] = (np.log(data[prediction]) - np.log(data[target]))**2
    RMSE = np.sqrt(sum(data.num) / len(data))
    RMSE
    return(RMSE)

Next chunk imports the leaderboard. Inspecting the scores shows 0.06626 is the score to beat, with most scores significantly under 1.

In [ ]:
# Import leaderboard
leaderboard = pd.read_csv("../Data/house-prices-advanced-regression-techniques-publicleaderboard.csv")

# Sort by score
leaderboard = leaderboard.sort_values('Score')
leaderboard.Score.describe()

In [ ]:
# Examine distribution of scores in first quartile
leaderboard[leaderboard.Score < 0.3].Score.hist(bins = 20)

## Prep Data for Modeling
When building predictive models, it is crucial that we split our data into *train* and *test* sets. This keeps our models honest as they are trained on one set of data, then tested on another. Later, we will talk about related concepts of overfitting and target leakage in relationship to these splits.

<img src="train_test_split.svg" width="600">

In [ ]:
# Split our data columns into features and target tables
X = home_data.drop(columns=['SalePrice', 'Id'])
y = home_data[['SalePrice']].astype('float')

In [ ]:
# Then split the rows into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1984)

print("Training Rows: {} \nTest Rows: {}".format(len(X_train), len(X_test)))

----------------------------------
## Linear Model
Our first approach will be a linear regression fit of home prices. This is similar to the type of fit we would perform in Minitab. Linear models are an excellent place to start due to their interpretability- we know exactly how predictions are being generated. 

On the other hand, linear models assume linear fits... an assumption that must be tested! Below we see an example of Anscombe's Quartet. From Wikipedia: 
*The data sets in the Anscombe's quartet are designed to have approximately the same linear regression line (as well as nearly identical means, standard deviations, and correlations) but are graphically very different. This illustrates the pitfalls of relying solely on a fitted model to understand the relationship between variables.*

Only the top-left quartet would be considered an appropriate model! 

<img src="AnscombesQuarter.png" width="600">

In [ ]:
# One-hot encode any categorical variables
def one_hot_mixed_df(data):
    data_cat = data.select_dtypes(include='object')
    data_num = data.select_dtypes(exclude='object')
    ## One-hot transform the objects
    data_cat_dummies = pd.get_dummies(data_cat,drop_first=True)
    ## Join dummies with numerics
    data_lm = pd.concat([data_cat_dummies, data_num], axis=1, sort=False)
    return(data_lm)

# Apply transformations
X_lm = one_hot_mixed_df(X)

## Verify results
print('Total Columns: {}'.format(X.shape[1]))
print('Total Columns post-Dummy: {}'.format(X_lm.shape[1]))

In [ ]:
print("Shape prior to NA cleanup: {}".format(X_lm.shape))

# NA values will crash the model fits, so let's see how to handle them
def count_NA(data):
    na_counts = data.isnull().sum(axis=0).reset_index(name='NACount')
    ## Print out the list of columns with missing values
    return(na_counts[na_counts.NACount>0])
    
print(count_NA(X_lm))

## We'll make a simplifying assumption that we can drop lot frontage and garage year built features. 
X_lm.drop(columns=['LotFrontage', 'GarageYrBlt'], inplace=True)

## For MasVnrArea, we will drop the NA rows. Note that rows also need to be dropped from our target!!!
row_drop_index =X_lm.MasVnrArea.notna()
X_lm = X_lm[row_drop_index]
y_lm = y[row_drop_index]

print("Shape of train after NA cleanup: {}".format(X_lm.shape))

In [ ]:
# Then split the rows into train and test
X_train, X_test, y_train, y_test = train_test_split(X_lm, y_lm, test_size=0.3, random_state=1984)

Let's finally fit the linear model to an object called "reg", then look at some information about the model.

In [ ]:
# Fit a linear model
reg = LinearRegression().fit(X_train, y_train)

# Print summaries
print("Score (Training):")
print(reg.score(X_train, y_train))
print("------------------------------------------------------------------")
print("Score (Test):")
print(reg.score(X_test, y_test))
print("------------------------------------------------------------------")
print("Intercept:")
print(reg.intercept_)
print("------------------------------------------------------------------")
print("Coefficients:")
print(reg.coef_)

Pause and reflect on a few questions:
* Why is our training score so much higher than testing?
* How might we improve our scores with a linear model?
    * Look at leverage and QQ-plots to identify points with high leverage and non-normal distributions. Feature selection and dimension reduction. 
* If we have a midling model, would you rely on the model weights to drive insights? Think about Anscombe's Quartet.

In [ ]:
clf = Ridge(alpha=1)
clf.fit(X_train, y_train)

# Print summaries
print("Score (Training):")
print(clf.score(X_train, y_train))
print("------------------------------------------------------------------")
print("Score (Test):")
print(clf.score(X_test, y_test))
print("------------------------------------------------------------------")
print("Intercept:")
print(clf.intercept_)
print("------------------------------------------------------------------")
print("Coefficients:")
print(clf.coef_)

Regularization imposes a penalty on having a lot of large weights. Notice how some of our ridge regression weights dropped to zero. Let's also look at the sum of our weights before and after regularization:

In [ ]:
print("Original Sum: {}".format(round(sum(reg.coef_[0]), 1)))
print("Regularized Sum: {}".format(round(sum(clf.coef_[0]),1)))

So let's count the ways this was a bad model:
1. We didn't test any assumptions of normality
2. No checks of colinearity- can affect model performance
3. Are the numeric features meaningful? i.e. Just because MSSubClass is a number (60, 70, 80...), doesn't mean math operations on those numbers are valid.
4. We have a LOT of columns given how few rows of data we have, and usually want 100 rows per column. We're about 21x short.

In [ ]:
# Calculate RMSE according to Kaggle method (RMSE between log-pred and log-act)
## Prep a dataframe to accept results
y_score = copy.deepcopy(y_test)
## Run predictions on test set
y_score['Pred_clf'] = clf.predict(X_test)
## Score RMSE
RMSE_clf = getRMSE(data=y_score, target='SalePrice', prediction='Pred_clf')
Percentile_clf = round(stats.percentileofscore(leaderboard.Score, RMSE_clf),1)
print("RMSE: {}".format(round(RMSE_clf, 5)))
print("Percentile on Leaderboard: {}%".format(Percentile_clf))

-----------------------------------
## Feature Engineering
We'll start by exporting data out to our IDEAR tool. This script streamlines common exploratory data analyses to help us find interesting patterns in the data.

In [ ]:
def ExportForIDEAR(data, target, filepath='idear_data.csv', verbose=False):
    data_cat = list(data.select_dtypes(include='object'))
    data_num = list(data.select_dtypes(exclude='object'))
#     data_num = [x for x in set(data_num) - set(target)]
    out_dict = {'CategoricalColumns': data_cat,
                'DataFilePath': filepath,
                'NumericalColumns': data_num,
                'Target': target}
    # Save yaml definition to file
    with open('IDEAR/data.yaml', 'w') as outfile:
        yaml.dump(out_dict, outfile, default_flow_style=False)
    
    # Write dataset to csv
    data.to_csv('IDEAR/'+filepath)
    
    # And prep output version for inspection
    if verbose:
        yaml_out = yaml.dump(out_dict, default_flow_style=False)
        return(yaml_out)

ExportForIDEAR(home_data, target='SalePrice')

### Cleanup
From our IDEAR analysis, we see an obvious need to clean the data. Let's start there.

In [ ]:
# Make a new dataframe to capture our transformations
engineered_Data = copy.deepcopy(home_data)
## Remove the ID column, this is an index with no predictive power
engineered_Data.drop(columns='Id', axis=1, inplace=True)

Clean up missing values:

In [ ]:
# NA Filler
def FillNACols(data, colname, value):
    data[colname][data[colname].isna()] = value
    return(data)

# Inspect NA counts
## Get the counts of NAs
na_counts = count_NA(engineered_Data)
## Print details about each column with NAs
for idx in na_counts['index']:
    print('Feature: {}'.format(idx))
    print('Number of NAs: {}'.format(na_counts.NACount[na_counts['index']==idx].item()))
    print(engineered_Data[idx].unique())
    print('==============================================')

# Fill values
## Simple fills
engineered_Data = FillNACols(engineered_Data, 'Alley', 'None')
engineered_Data = FillNACols(engineered_Data, 'PoolQC', 'None')
engineered_Data = FillNACols(engineered_Data, 'MiscFeature', 'None')
engineered_Data = FillNACols(engineered_Data, 'Fence', 'None')
engineered_Data = FillNACols(engineered_Data, 'FireplaceQu', 'None')
engineered_Data = FillNACols(engineered_Data, 'GarageFinish', 'None')
engineered_Data = FillNACols(engineered_Data, 'GarageCond', 'None')
engineered_Data = FillNACols(engineered_Data, 'GarageQual', 'None')
engineered_Data = FillNACols(engineered_Data, 'GarageType', 'None')
engineered_Data = FillNACols(engineered_Data, 'BsmtExposure', 'None')
engineered_Data = FillNACols(engineered_Data, 'BsmtQual', 'None')
engineered_Data = FillNACols(engineered_Data, 'BsmtCond', 'None')
engineered_Data = FillNACols(engineered_Data, 'BsmtFinType1', 'None')
engineered_Data = FillNACols(engineered_Data, 'BsmtFinType2', 'None')

## Remove Rows
engineered_Data = engineered_Data = engineered_Data[pd.notna(engineered_Data['MasVnrType'])]
engineered_Data = engineered_Data = engineered_Data[pd.notna(engineered_Data['Electrical'])]

## Remove Columns
cols_to_drop = ['GarageYrBlt', 'LotFrontage']
try:
    engineered_Data.drop(columns=cols_to_drop, inplace=True)
except:
    print("Columns already dropped")

Apply power transformations to non-normal distributions, getting them closer to Gaussian.

In [ ]:
# Power transforms
def log_transform(data):
    data = np.log(data)
    return(data)

def reverse_log_transform(data):
    data = np.exp(data)
    return(data)
    
# List columns to transform
cols_to_pt = ['SalePrice',
              '1stFlrSF',
              '2ndFlrSF',
              'GrLivArea',
              'LotArea']

## Add a little noise before transforming
engineered_Data[cols_to_pt] = engineered_Data[cols_to_pt] + 0.01
engineered_Data[cols_to_pt] = log_transform(engineered_Data[cols_to_pt])

Some categories have a lot of levels, many of which have few samples. We will want to clean these up as well.

Let's have a look at what our cleanup does to our model. 
Note that we reduce RMSE by 0.01, or ~6%, which is good enough for a 9% jump on the leaderboard.

In [ ]:
def kaggle_score(RMSE_score, leaderboard):
    Percentile = round(stats.percentileofscore(leaderboard.Score, RMSE_score), 1)
    Rank = float(sum([x < RMSE_score for x in leaderboard.Score]))
    print("RMSE: {}".format(round(RMSE_score, 5)))
    print("Percentile on Leaderboard: {}%".format(Percentile))
    print("Leaderboard Rank: {} of {}".format(int(Rank), len(leaderboard)))

# Function to evaluate feature performance on regularized LM
def evaluate_model(data):
    # Split our data columns into features and target tables
    X = data.drop(columns=['SalePrice'])
    y = data[['SalePrice']].astype('float')
    # Apply transformations
    X_lm = one_hot_mixed_df(X)
    # Then split the rows into train and test
    X_train, X_test, y_train, y_test = train_test_split(X_lm, y, test_size=0.3, random_state=1984)

    clf = Ridge(alpha=1)
    clf.fit(X_train, y_train)

    # Calculate RMSE according to Kaggle method (RMSE between log-pred and log-act)
    ## Prep a dataframe to accept results
    y_score = copy.deepcopy(y_test)
    ## Run predictions on test set
    y_score['Pred_clf'] = clf.predict(X_test)
    ## Inverse log to get back to real prices
    y_score = reverse_log_transform(y_score)

    ## Score RMSE
    RMSE_clf = getRMSE(data=y_score, target='SalePrice', prediction='Pred_clf')
#     Percentile_clf = round(stats.percentileofscore(leaderboard.Score, RMSE_clf),1)
#     print("RMSE: {}".format(round(RMSE_clf, 5)))
#     print("Percentile on Leaderboard: {}%".format(Percentile_clf))
    kaggle_score(RMSE_clf, leaderboard)

# Evaluate current configuration 
evaluate_model(engineered_Data)

### Synthetic Features

All we've done so far is janitorial work. Feature engineering is also the step where we start to incorporate subject matter expertise to synthesize inputs that add significant performance. Some ideas include:
* Adjust the lot size per neighborhood. Some neighborhoods are more spacious, so maybe the relative size is more important than absolute square feet?
* From PCA, we see that some homes show tell-tale signs of being rentals with many people (e.g. multiple kitchens, high count of beds and baths). Try setting a flag for such properties.
* When house shopping, one strategy for home valuation is to study the multiplier between tax assessments and sale prices of homes in a neighborhood, then applying this scaling to the tax assessment of a property. How can we do this?

#### Taxes
Let's visit the Ames, Iowa city assessor: [LINK](https://www.cityofames.org/government/departments-divisions-a-h/city-assessor)

Under "Reports", it looks like we can find what we need. Luckily, this report is already downloaded to our data folder. What's in there?

In [ ]:
tax_Data = pd.read_excel('../Data/AmesRealEstateData.xlsx')

In [ ]:
# Rename some columns to match our dataset
tax_Data.rename(index=str, columns={"GLA": "GrLivArea", "YrBuilt": "YearBuilt"}, inplace=True)
tax_Data.head()

# Remove incomplete tax records
tax_Data = tax_Data[(tax_Data.GrLivArea.notna()) & (tax_Data.LotArea.notna()) & (tax_Data.YearBuilt.notna())]

In [ ]:
print("  |  ".join(list(tax_Data)))

Unfortunately, our training data doesn't list addresses or lot ids of any kind. We need to get creative to combine this data. Instead of a clear "key" (i.e. address) to join our data, we need to generate a synthetic key based on identifying features. 

Inspecting the overlap of columns below, we see many shared features that are candidates for joining. Bear in mind, however, that some features (like Roofing material) are subject to change over time and may not be as reliable as lot size, for example.

In [ ]:
# List overlapping columns
print(set(list(tax_Data)) & set(list(engineered_Data)))

Below, we see that many records match outright. While this is pretty good, we need significantly more coverage in order to use this feature.

In [ ]:
syn_key = ['GrLivArea', 'YearBuilt', 'LotArea', 'Neighborhood']

# Try an inner join to see how many records line up
print("Sample records in dataset: {}".format(len(engineered_Data)))

record_overlap = home_data.merge(tax_Data, on=syn_key, how='inner').drop_duplicates()
print("Overlapping records: {}".format(len(record_overlap.Id.unique())))

In [ ]:
# Investigate neighborhood naming
print("Neighborhoods in home data missing from tax data")
print(list(set(home_data.Neighborhood.unique()) - set(tax_Data.Neighborhood.unique())))

print("Home Data:")
print(list(set(home_data.Neighborhood.unique())))
print("Tax Data:")
print(list(set(tax_Data.Neighborhood.unique())))

In [ ]:
# Looks like we need to clean up. Drop whitespace and special characters
home_data['Neighborhood'] = home_data['Neighborhood'].str.strip()
tax_Data['Neighborhood'] = tax_Data['Neighborhood'].str.strip()
tax_Data['Neighborhood'] = tax_Data['Neighborhood'].str.replace('\W', '')

tax_Data.loc[tax_Data.Neighborhood=='Bluestm', 'Neighborhood'] = 'Blueste'
tax_Data.loc[tax_Data.Neighborhood=='NoPkVil', 'Neighborhood'] = 'NPkVill'
tax_Data.loc[tax_Data.Neighborhood=='Stonebr', 'Neighborhood'] = 'StoneBr'
tax_Data.loc[tax_Data.Neighborhood=='NRidgHt', 'Neighborhood'] = 'NridgHt'
tax_Data.loc[tax_Data.Neighborhood.isna(), 'Neighborhood'] = 'None'
print("Remaining Neighborhood Mismatches:")
print(print(list(set(home_data.Neighborhood.unique()) - set(tax_Data.Neighborhood.unique()))))

To improve matches, we'll use a distance-based measure to match home records to their closest tax record counterpart. This will maintain perfect matches (i.e. distance == 0), but will accomodate records with imperfect matches. In addition to compensating for changes in a home's definition, it also effectively matches the record to the nearest comparable.

In [ ]:
from sklearn.preprocessing import StandardScaler

def match_score(taxdf, targetRecord, keys):
    taxdf = taxdf.loc[:, keys + ['Prop_Addr', 'TtlVal_AsrYr']]
    targetRecord = targetRecord.loc[:, keys]
    
    # List numeric columns
    num_keys = list(targetRecord.loc[:, keys].select_dtypes(include='number'))
    # List object columns
    obj_keys = list(targetRecord.loc[:, keys].select_dtypes(include='object'))
    
    # Measure errors
    for key in num_keys:
        # Measure error
        taxdf['Error{}'.format(key)] = np.abs(taxdf[key] - targetRecord[key].item())
        # Scale
        scaler = StandardScaler(with_mean=False)
        taxdf['Error{}'.format(key)] = scaler.fit_transform(np.array(taxdf['Error{}'.format(key)]).reshape(-1,1))

    for key in obj_keys:
        # Measure error- 0 if match, 1 if not matched
        taxdf['Error{}'.format(key)] = taxdf[key] != targetRecord[key].item()
        taxdf['Error{}'.format(key)] = taxdf['Error{}'.format(key)].astype('int') * 5
        
    # Distance measure
    taxdf['Similarity'] = taxdf[taxdf.columns[pd.Series(taxdf.columns).str.startswith('Error')]].sum(axis=1)
    
    # Re-arrange
    taxdf = taxdf.sort_values('Similarity', ascending=True)
    return(taxdf.iloc[0,:]['TtlVal_AsrYr'])

In [ ]:
# Generate a list of tax assessments using our matching function
tax_assessments = [match_score(tax_Data, engineered_Data.iloc[x:x+1, :], syn_key) for x in range(0, len(engineered_Data))]
# Join the list to our dataset
engineered_Data['tax_assessment'] = tax_assessments
engineered_Data.head()

Let's reflect on what we've done here- we've taken an anonymized data set, and de-anonymized it. In this case, the "anonymous" data used in the competition is publicly available and was only witheld for competition purposes, but this raises a couple of interesting questions:
* Even if we anonymize our data, can it be reconstituted? Is that a problem?
* In the case of the competition, we *could* now simply join published SalePrices and have *perfect* predictions. We'd win the competition, but how predictive is this model? This is called target leakage, and it's a serious issue because it's not always so obvious.

#### Tax Multiplier


In [ ]:
# Temporarily place a non-logged saleprice column
engineered_Data['SalePrice_NoLog'] = np.exp(engineered_Data['SalePrice'])
# Calculate neighborhood means
nbd_tax_mult = engineered_Data.groupby('Neighborhood').agg({'SalePrice_NoLog': 'mean', 'tax_assessment': 'mean'})
# Calculate ratios
nbd_tax_mult['tax_mult'] = nbd_tax_mult['SalePrice_NoLog'] / nbd_tax_mult['tax_assessment']
nbd_tax_mult.drop(columns=['SalePrice_NoLog', 'tax_assessment'], inplace=True)

# Join back into engineered_Data
engineered_Data = engineered_Data.merge(nbd_tax_mult, on='Neighborhood', how='left')

In [ ]:
# Calculate scaled tax assessment
engineered_Data['scaled_tax_assessment'] = engineered_Data['tax_assessment'] * engineered_Data['tax_mult']

# Clean up the temp columns
engineered_Data.drop(columns=['SalePrice_NoLog', 'tax_mult'], inplace=True)

engineered_Data.head()

#### Relative Lot Size


In [ ]:
# Get mean lot area by neighborhood
lot_mult = engineered_Data.groupby('Neighborhood').agg({'LotArea': 'mean'})
lot_mult.rename(index=str, columns={"LotArea": "MeanNBDLotArea"}, inplace=True)
# Join with dataset
engineered_Data = engineered_Data.merge(lot_mult, on='Neighborhood', how='left')

In [ ]:
# Calculate individual lot size ratios
engineered_Data['RelativeLotArea'] = engineered_Data['LotArea'] / engineered_Data['MeanNBDLotArea']

# Clean up the temp columns
engineered_Data.drop(columns=['MeanNBDLotArea'], inplace=True)

## Machine Learning

Whereas linear models represent patterns in our data with straight lines/planes, machine learning methods can fit non-linear shapes to our data (see example below). As we travel down this path, we trade interpretability for performance. Some methods are more interpretable (decision trees), while others discard interpretability altogether for maximum performance (neural networks).

Note that interpretability is relative. While it is nearly impossible to look at a neural network and make any sense of it, we can use sensitivity analysis to understand how it arrived at a recommendation.

<img src="nonlinear_fits.gif" width="500">

In [ ]:
# Split our data columns into features and target tables
X = engineered_Data.drop(columns=['SalePrice'])
y = engineered_Data[['SalePrice']].astype('float')
# Apply transformations
X_lm = one_hot_mixed_df(X)
# Then split the rows into train and test
X_train, X_test, y_train, y_test = train_test_split(X_lm, y, test_size=0.3, random_state=1984)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import xgboost as xgb


run_cv = False

if run_cv:
    # Generate a subsample for quicker training
    sub_idx = np.random.randint(0, len(y_train), int(len(y_train)*0.99))
    X_train_cv = X_train.iloc[sub_idx,:]
    y_train_cv = y_train.iloc[sub_idx,:]

    xg_reg = xgb.XGBRegressor(objective ='reg:linear',
                              learning_rate = 0.3,
                              n_estimators = 25)

    colsample_bytree = [0.3, 0.6, 0.8]
    max_depth = [2, 4, 6, 8]
    alpha = [1, 3, 8]
    subsample = [0.3, 0.6, 0.8]
    param_grid = dict(colsample_bytree=colsample_bytree,
                      max_depth=max_depth,
                      reg_alpha=alpha,
                      subsample=subsample)

    kfold = KFold(n_splits=10, shuffle=True, random_state=7)
    grid_search = GridSearchCV(xg_reg, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, cv=kfold, verbose=1)
    grid_result = grid_search.fit(X_train_cv, y_train_cv)

    print("Best Score (MSE): {} \nParams: {}".format(grid_result.best_score_, grid_result.best_params_))

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear',
                          colsample_bytree = 0.8,
                          learning_rate = 0.1,
                          max_depth = 6,
                          subsample=0.8,
                          alpha = 1,
                          n_estimators = 200,
                         seed=555111)


# Train model
xgbData = xgb.DMatrix(X_train, label=y_train)
eval_set = [(X_train, y_train), (X_test, y_test)]
xg_reg.fit(X_train, y_train, eval_metric="rmse", eval_set=eval_set, verbose=True)

First, we will have a look at the learning curves for this model. The curves provide a lot of good feedback for model training.
<img src="biasvariance.png" width="500">

In [ ]:
results = xg_reg.evals_result()
epochs = len(results['validation_0']['rmse'])
x_axis = range(0, epochs)

fig, ax = plt.subplots(figsize=(10,7))
ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
ax.plot(x_axis, results['validation_1']['rmse'], label='Test')
ax.legend()
plt.ylabel('RMSE')
plt.title('XGBoost RMSE Learning Curves')
plt.ylim(0,0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,35))
xgb.plot_importance(xg_reg, ax=ax)
plt.show()

In [ ]:
def residual_plots(df, sample_fraction):
    # Subsample plot data
    df = df.sample(frac=sample_fraction, replace=False)
    
    # Residual plot
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 12))
    ax1.scatter('pred', 'residual', data=df, alpha=1, s=3)
    ax1.axhline(y=0, c='k')
    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('Residual')

    # Pred-actual plot
    ax2.scatter('y', 'pred', data=df, alpha=1, s=3)
    ax2.plot((11,13), (11,13), ls="--", c=".3")
    ax2.set_xlabel('Actual')
    ax2.set_ylabel('Predicted')
    
    
    # Histogram of residuals
    ax3.hist(df['residual'], bins = 20)
    ax3.set_xlabel('Residual')
    
    # Distributions of salaries
    sns.kdeplot(df['pred'], ax=ax4)
    sns.kdeplot(df['y'], ax=ax4)
    
    plt.show()

In [ ]:
from sklearn import metrics

preds = xg_reg.predict(X_test)

# Inverse transformation if using a transformed target variable
y_measures = y_test
# Prepare plot data
plot_data = pd.DataFrame({'pred': preds.tolist()})
plot_data['y'] = y_measures.reset_index()['SalePrice']


# Calculate residuals
plot_data['residual'] = plot_data['pred'] - plot_data['y']

# calculate rmse
mse = metrics.mean_squared_error(y_measures, preds)  
rmse = np.sqrt(mse)
print('RMSE: {}'.format(rmse))
# calculate mae
mae = metrics.mean_absolute_error(y_measures, preds)  
print('MAE: {}'.format(mae))


# Plot residuals    
residual_plots(plot_data, 0.5)

In [ ]:
kaggle_score(RMSE_score=rmse, leaderboard=leaderboard)

# Examine distribution of scores in first quartile
leaderboard[leaderboard.Score < 0.3].Score.hist(bins = 30)